<a href="https://colab.research.google.com/github/MBrandao07/Prevencao-a-Fraudes/blob/main/Codigos/3_Preven%C3%A7%C3%A3o_a_Fraude_Modelo_Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelagem

In [ ]:
!pip install optuna
!pip install category_encoders
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from category_encoders import TargetEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score, ShuffleSplit
from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, \
HistGradientBoostingClassifier,StackingClassifier
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, roc_auc_score, f1_score, make_scorer, accuracy_score, precision_score, recall_score, auc
import optuna
import pickle
from sklearn.ensemble import VotingClassifier, StackingClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# conectando ao drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Funções auxiliares

In [ ]:
# Métricas do modelo

def calculate_metrics(nm_modelo, model, X_train, y_train, X_test, y_test):
    # Fazendo predições
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Calculando as métricas para o conjunto de treino
    accuracy_train = accuracy_score(y_train, y_train_pred)
    precision_train = precision_score(y_train, y_train_pred)
    recall_train = recall_score(y_train, y_train_pred)
    auc_roc_train = roc_auc_score(y_train, model.predict_proba(X_train)[:, 1])

    # Calculando o Índice Gini e Estatística KS para o conjunto de treino
    probabilities_train = model.predict_proba(X_train)[:, 1]
    df_train = pd.DataFrame({'true_labels': y_train, 'predicted_probs': probabilities_train})
    df_train = df_train.sort_values(by='predicted_probs', ascending=False)
    df_train['cumulative_true'] = df_train['true_labels'].cumsum() / df_train['true_labels'].sum()
    df_train['cumulative_false'] = (1 - df_train['true_labels']).cumsum() / (1 - df_train['true_labels']).sum()
    ks_statistic_train = max(abs(df_train['cumulative_true'] - df_train['cumulative_false']))
    gini_index_train = 2 * auc_roc_train - 1

    # Calculando as métricas para o conjunto de teste
    accuracy_test = accuracy_score(y_test, y_test_pred)
    precision_test = precision_score(y_test, y_test_pred)
    recall_test = recall_score(y_test, y_test_pred)
    auc_roc_test = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])

    # Calculando o Índice Gini e Estatística KS para o conjunto de teste
    probabilities_test = model.predict_proba(X_test)[:, 1]
    df_test = pd.DataFrame({'true_labels': y_test, 'predicted_probs': probabilities_test})
    df_test = df_test.sort_values(by='predicted_probs', ascending=False)
    df_test['cumulative_true'] = df_test['true_labels'].cumsum() / df_test['true_labels'].sum()
    df_test['cumulative_false'] = (1 - df_test['true_labels']).cumsum() / (1 - df_test['true_labels']).sum()
    ks_statistic_test = max(abs(df_test['cumulative_true'] - df_test['cumulative_false']))
    gini_index_test = 2 * auc_roc_test - 1

    # Criando o DataFrame com as métricas calculadas
    metrics_df = pd.DataFrame({
        'Algoritmo': [nm_modelo, nm_modelo],
        'Conjunto': ['Treino', 'Teste'],
        'Acuracia': [accuracy_train, accuracy_test],
        'Precisao': [precision_train, precision_test],
        'Recall': [recall_train, recall_test],
        'AUC_ROC': [auc_roc_train, auc_roc_test],
        'GINI': [gini_index_train, gini_index_test],
        'KS': [ks_statistic_train, ks_statistic_test]
    })
    return metrics_df

def calculate_metrics_train(nm_modelo, X_train, y_train):
    # Fazendo predições
    y_train_prob = X_train.score_fraude
    y_train_pred = (y_train_prob > 0.5).astype(int)

    # Calculando as métricas para o conjunto de treino
    accuracy_train = accuracy_score(y_train, y_train_pred)
    precision_train = precision_score(y_train, y_train_pred)
    recall_train = recall_score(y_train, y_train_pred)
    auc_roc_train = roc_auc_score(y_train, y_train_prob)

    # Calculando o Índice Gini e Estatística KS para o conjunto de treino
    probabilities_train = y_train_prob
    df_train = pd.DataFrame({'true_labels': y_train, 'predicted_probs': probabilities_train})
    df_train = df_train.sort_values(by='predicted_probs', ascending=False)
    df_train['cumulative_true'] = df_train['true_labels'].cumsum() / df_train['true_labels'].sum()
    df_train['cumulative_false'] = (1 - df_train['true_labels']).cumsum() / (1 - df_train['true_labels']).sum()
    ks_statistic_train = max(abs(df_train['cumulative_true'] - df_train['cumulative_false']))
    gini_index_train = 2 * auc_roc_train - 1

    # Criando o DataFrame com as métricas calculadas
    metrics_df = pd.DataFrame({
        'Algoritmo': [nm_modelo],
        'Conjunto': ['Treino'],
        'Acuracia': [accuracy_train],
        'Precisao': [precision_train],
        'Recall': [recall_train],
        'AUC_ROC': [auc_roc_train],
        'GINI': [gini_index_train],
        'KS': [ks_statistic_train]
    })
    return metrics_df

def plot_metrics(model, X_train, y_train, X_test, y_test, features):
    plt.figure(figsize=(15, 18))

    # Gráfico de Barras para Taxa do Evento e Importância das Variáveis
    plt.subplot(4, 2, 1)
    event_rate = y_train.mean()
    non_event_rate = 1 - event_rate
    plt.barh(['Não Evento', 'Evento'], [non_event_rate, event_rate], color=['lightsteelblue', 'skyblue'])
    plt.xlabel('Taxa')
    plt.title('Taxa do Evento')

    plt.subplot(4, 2, 2)
    importances = model.feature_importances_
    indices = np.argsort(importances)  # Ordem ascendente
    indices = indices[-10:]  # Considerando apenas as 10 mais importantes
    plt.barh(features[indices], importances[indices], color='skyblue')
    plt.xlabel('Importância')
    plt.title('Importância das Variáveis')

    # Curva ROC para Treino e Teste
    plt.subplot(4, 2, 3)
    for X, y, label in [(X_train, y_train, 'Treino'), (X_test, y_test, 'Teste')]:
        fpr, tpr, _ = roc_curve(y, model.predict_proba(X)[:, 1])
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f'{label} (AUC = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Curva ROC')
    plt.legend(loc='lower right')

    # Gráfico de Barras para Gini e KS
    plt.subplot(4, 2, 4)
    metrics = ['Índice Gini', 'Estatística KS']
    for X, y, label in [(X_train, y_train, 'Treino'), (X_test, y_test, 'Teste')]:
        probs = model.predict_proba(X)[:, 1]
        df = pd.DataFrame({'true_labels': y, 'predicted_probs': probs})
        df = df.sort_values(by='predicted_probs', ascending=False)
        df['cumulative_true'] = df['true_labels'].cumsum() / df['true_labels'].sum()
        df['cumulative_false'] = (1 - df['true_labels']).cumsum() / (1 - df['true_labels']).sum()
        ks = max(abs(df['cumulative_true'] - df['cumulative_false']))
        gini = 2 * auc(*roc_curve(y, probs)[:2]) - 1
        plt.bar([f'{label}\n{metric}' for metric in metrics], [gini, ks], color='skyblue')
    plt.title('Índice Gini e Estatística KS')

    # Matrizes de Confusão para Treino e Teste
    for i, (X, y, label) in enumerate([(X_train, y_train, 'Treino'), (X_test, y_test, 'Teste')], 5):
        plt.subplot(4, 2, i)
        conf_matrix = confusion_matrix(y, model.predict(X))
        sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar_kws={'label': 'Count'})
        plt.xlabel('Predicted Label')
        plt.ylabel('True Label')
        plt.title(f'Matriz de Confusão - {label}')

    # Histograma do Score para Treino e Teste
    plt.subplot(4, 2, 7)
    plt.hist(model.predict_proba(X_train)[:, 1], bins=30, color='skyblue', edgecolor='black', alpha=0.7, label='Treino')
    plt.xlabel('Score')
    plt.ylabel('Frequência')
    plt.title('Histograma do Score - Treino')
    plt.legend(loc='upper right')

    plt.subplot(4, 2, 8)
    plt.hist(model.predict_proba(X_test)[:, 1], bins=30, color='lightsteelblue', edgecolor='black', alpha=0.7, label='Teste')
    plt.xlabel('Score')
    plt.ylabel('Frequência')
    plt.title('Histograma do Score - Teste')
    plt.legend(loc='upper right')

    plt.tight_layout()
    plt.show()

### Importando os datasets

In [ ]:
# base treino
df_train = pd.read_csv('/content/drive/MyDrive/2 - Projetos Portfolio/Prevencao de Fraude/Base de dados/df_train_full.csv')

# base teste
df_test = pd.read_csv('/content/drive/MyDrive/2 - Projetos Portfolio/Prevencao de Fraude/Base de dados/df_test_full.csv')

In [ ]:
df_train.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_FRAUD,x_customer_id,y_customer_id,mean_amount,std_amount,...,RATIO_MEAN_AMOUNT_PER_CUSTOMER_LAST_1H_TO_8H,RATIO_MEAN_AMOUNT_PER_CUSTOMER_LAST_1H_TO_24H,RATIO_MEAN_AMOUNT_PER_CUSTOMER_LAST_1H_TO_7D,RATIO_MEAN_AMOUNT_PER_CUSTOMER_LAST_1H_TO_14D,RATIO_TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_1H_TO_2H,RATIO_TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_1H_TO_4H,RATIO_TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_1H_TO_8H,RATIO_TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_1H_TO_24H,RATIO_TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_1H_TO_7D,RATIO_TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_1H_TO_14D
0,59452,2021-08-01 03:01:00,0,1133,61.51,0,10.95017,59.768684,62.262521,31.13126,...,1.000000,1.000000,1.000000,1.000000,1.0,1.0,1.000000,1.000000,1.000000,1.000000
1,59592,2021-08-01 05:30:38,0,1138,129.61,0,10.95017,59.768684,62.262521,31.13126,...,1.356321,1.356321,1.356321,1.356321,1.0,0.5,0.500000,0.500000,0.500000,0.500000
2,60132,2021-08-01 10:40:12,0,1530,96.50,0,10.95017,59.768684,62.262521,31.13126,...,1.006536,1.006536,1.006536,1.006536,1.0,1.0,0.333333,0.333333,0.333333,0.333333
3,60721,2021-08-01 15:38:25,0,241,82.19,0,10.95017,59.768684,62.262521,31.13126,...,0.919917,0.888997,0.888997,0.888997,1.0,1.0,0.500000,0.250000,0.250000,0.250000
4,61033,2021-08-01 19:25:30,0,1536,50.77,0,10.95017,59.768684,62.262521,31.13126,...,0.763688,0.603571,0.603571,0.603571,1.0,0.5,0.500000,0.200000,0.200000,0.200000


## Preparação dos dados para o modelo

In [ ]:
# dropando as variáveis que não fazem sentido
X = df_train.drop(columns=['TX_FRAUD','TRANSACTION_ID',	'TX_DATETIME',	'CUSTOMER_ID',	'TERMINAL_ID','x_customer_id','y_customer_id','x_terminal_id','y_terminal_id'])

# isolando a variável resposta
y = df_train['TX_FRAUD']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

cat_attributes = X_train.select_dtypes(include='object')
num_attributes = X_train.select_dtypes(exclude='object')

# definindo a pipeline para as variáveis categóricas
cat_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', TargetEncoder())
])

# definindo a pipeline para as variáveis numéricas
num_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler',StandardScaler())
])

# Combinando as pipelines
preprocessor = ColumnTransformer([
    ('cat', cat_pipe, cat_attributes.columns),
    ('num', num_pipe, num_attributes.columns)
])
preprocesssor = Pipeline(steps=[("preprocessor", preprocessor)])

X_train_processed = pd.DataFrame(preprocesssor.fit_transform(X_train, y_train),columns=X_train.columns)
X_test_processed = pd.DataFrame(preprocesssor.transform(X_test),columns=X_test.columns)

In [ ]:
X_train_processed.head()

,TX_AMOUNT,mean_amount,std_amount,mean_nb_tx_per_day,month_reference,PERIODO_DIA,hour_of_day,day_of_week,day_of_month,month,...,RATIO_MEAN_AMOUNT_PER_CUSTOMER_LAST_1H_TO_8H,RATIO_MEAN_AMOUNT_PER_CUSTOMER_LAST_1H_TO_24H,RATIO_MEAN_AMOUNT_PER_CUSTOMER_LAST_1H_TO_7D,RATIO_MEAN_AMOUNT_PER_CUSTOMER_LAST_1H_TO_14D,RATIO_TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_1H_TO_2H,RATIO_TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_1H_TO_4H,RATIO_TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_1H_TO_8H,RATIO_TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_1H_TO_24H,RATIO_TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_1H_TO_7D,RATIO_TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_1H_TO_14D
0,0.022084,0.088162,-0.199866,0.271800,0.271800,-0.215334,0.715392,-0.300093,1.011665,1.286730,...,-0.210047,-0.554798,-0.440023,-0.428959,-1.628568,-1.363005,-1.058727,-0.651352,-0.031440,0.124756
1,0.023691,0.018921,2.311177,1.604511,1.604511,-0.077645,0.715392,1.280991,1.011665,1.286730,...,0.177343,0.713260,0.790510,1.013735,0.379626,0.661940,-0.717724,-0.651352,-0.273222,-0.216624
2,0.023691,0.015691,0.045859,-0.481662,-0.481662,0.962510,-1.406686,1.280991,-1.486936,-0.765401,...,0.782273,0.386903,0.663435,0.645064,-3.636762,-2.375478,-1.740734,-1.061308,-0.524675,-0.279618
3,0.022084,0.046411,0.065558,0.137224,0.137224,0.579580,1.422752,-0.695364,0.511945,0.146657,...,0.002985,-0.271210,-0.401869,-0.442176,0.379626,0.661940,0.987294,-0.514700,-0.173174,-0.159073
4,0.022084,0.019228,1.118413,0.768109,0.768109,-0.607464,1.422752,-0.497729,1.011665,0.260664,...,0.710735,0.752350,0.685118,0.535742,0.379626,-1.363005,-0.717724,-0.842664,-0.334304,-0.261020


## Modelagem

### Treinando diversos algoritmos diferentes para obter o melhor resultado

In [ ]:
algoritmos = [DecisionTreeClassifier(criterion='gini',random_state=0),
            LogisticRegression(solver='liblinear',random_state=0),
            RandomForestClassifier(random_state=0),
            GradientBoostingClassifier(random_state=0),
            XGBClassifier(random_state=0),
            lgb.LGBMClassifier(random_state=0)]

for algoritmo in algoritmos:

    nome_algoritmo = str(algoritmo)[:str(algoritmo).find("(")]
    # Treino do modelo
    algoritmo.fit(X_train_processed,y_train)

    # Avaliar modelo
    metricas = calculate_metrics(nome_algoritmo,algoritmo, X_train_processed, y_train, X_test_processed, y_test)
    display(metricas)

,Algoritmo,Conjunto,Acuracia,Precisao,Recall,AUC_ROC,GINI,KS
0,DecisionTreeClassifier,Treino,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000
1,DecisionTreeClassifier,Teste,0.964736,0.092606,0.06629,0.525747,0.051493,0.054601


,Algoritmo,Conjunto,Acuracia,Precisao,Recall,AUC_ROC,GINI,KS
0,LogisticRegression,Treino,0.983950,0.872271,0.344694,0.981538,0.963077,0.882185
1,LogisticRegression,Teste,0.974465,0.240437,0.067831,0.669950,0.339900,0.274811


,Algoritmo,Conjunto,Acuracia,Precisao,Recall,AUC_ROC,GINI,KS
0,RandomForestClassifier,Treino,0.999990,1.000000,0.999569,1.000000,1.000000,1.000000
1,RandomForestClassifier,Teste,0.975678,0.304158,0.071429,0.697208,0.394415,0.328967


,Algoritmo,Conjunto,Acuracia,Precisao,Recall,AUC_ROC,GINI,KS
0,GradientBoostingClassifier,Treino,0.985892,0.857433,0.455349,0.987889,0.975778,0.915279
1,GradientBoostingClassifier,Teste,0.973870,0.202822,0.059096,0.601617,0.203233,0.227914


,Algoritmo,Conjunto,Acuracia,Precisao,Recall,AUC_ROC,GINI,KS
0,XGBClassifier,Treino,0.996370,0.989693,0.849223,0.999461,0.998921,0.979019
1,XGBClassifier,Teste,0.973229,0.203620,0.069373,0.709589,0.419177,0.331020


[LightGBM] [Info] Number of positive: 4636, number of negative: 199225
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.371913 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 21435
[LightGBM] [Info] Number of data points in the train set: 203861, number of used features: 109
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.022741 -> initscore=-3.760583
[LightGBM] [Info] Start training from score -3.760583


,Algoritmo,Conjunto,Acuracia,Precisao,Recall,AUC_ROC,GINI,KS
0,LGBMClassifier,Treino,0.990547,0.940774,0.623598,0.997321,0.994642,0.960915
1,LGBMClassifier,Teste,0.973641,0.210697,0.066804,0.639679,0.279359,0.212544


### Avaliando a performance do modelo vigente

In [ ]:
abt_vigente = pd.read_csv('/content/drive/MyDrive/2 - Projetos Portfolio/Prevencao de Fraude/Base de dados/abt_vigente_score.csv')

In [ ]:
metricas_vigente = calculate_metrics_train('Modelo Vigente', abt_vigente,y)
display(metricas_vigente)

,Algoritmo,Conjunto,Acuracia,Precisao,Recall,AUC_ROC,GINI,KS
0,Modelo Vigente,Treino,0.023596,0.0226,0.998936,0.500947,0.001894,0.005815


### Salvando o melhor modelo obtido

In [ ]:
import joblib

# Definindo qual foi o melhor modelo
melhor_modelo = XGBClassifier(random_state=0)
melhor_modelo.fit(X_train_processed, y_train)

# Salvar o modelo em disco
joblib.dump(melhor_modelo, '/content/drive/MyDrive/2 - Projetos Portfolio/Prevencao de Fraude/Modelos/modelo_baseline.pkl')

['/content/drive/MyDrive/2 - Projetos Portfolio/Prevencao de Fraude/Modelos/modelo_baseline.pkl']

In [ ]:
# carregando o modelo novamente
modelo_carregado = joblib.load('/content/drive/MyDrive/2 - Projetos Portfolio/Prevencao de Fraude/Modelos/modelo_baseline.pkl')

In [ ]:
# testando o modelo
y_pred = modelo_carregado.predict(X_test_processed)
y_pred

array([0, 0, 0, ..., 0, 0, 0])